<a href="https://colab.research.google.com/github/leonardo3108/robustez-query/blob/main/Teste%20Marco%20Passage%20%2B%20BM25%20%2B%20Rerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/castorini/pygaggle.git

  Cloning https://github.com/castorini/pygaggle.git to /tmp/pip-req-build-_80r9u1q
  Running command git clone -q https://github.com/castorini/pygaggle.git /tmp/pip-req-build-_80r9u1q
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 9.1 MB 1.0 MB/s 
     |████████████████████████████████| 72.8 MB 1.2 MB/s 
     |████████████████████████████████| 22.3 MB 62.3 MB/s 
     |████████████████████████████████| 25.9 MB 61.8 MB/s 
     |████████████████████████████████| 12.8 MB 16.2 MB/s 
     |████████████████████████████████| 6.0 MB 27.4 MB/s 
     |████████████████████████████████| 454.3 MB 16 kB/s 
     |████████████████████████████████| 3.3 MB 43.5 MB/s 
     |████████████████████████████████| 72 kB 1.0 MB/s 
     |████████████████████████████████| 2.2 MB 36.5 MB/s 
     |████████████████████████████████| 1.2 MB 41.8 MB/s 
     |████████████████████████████████| 85 kB 4.8 MB/s 
     

In [2]:
!pip install pyserini

In [3]:
from pyserini.search import SimpleSearcher

# `SimpleSearcher` defaults to BM25 scoring function.
searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')

index-msmarco-passage-20201117-f87c94.tar.gz: 0.00B [00:00, ?B/s]

Attempting to initialize pre-built index msmarco-passage.


index-msmarco-passage-20201117-f87c94.tar.gz: 2.07GB [01:12, 30.8MB/s]                            


Extracting /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.tar.gz into /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d...
Initializing msmarco-passage...


In [6]:
import json
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.base import hits_to_texts

# Here's our query:
query = Query('who proposed the geocentric theory')

hits = searcher.search(query.text, k=10)
texts = hits_to_texts(hits)

print('BM25:')
for i, text in enumerate(texts):
    content = json.loads(text.text)['contents']
    print(f'{i+1:2} {text.metadata["docid"]:15} {text.score:.5f} {content}')

BM25:
 1 7744105         14.35620 For Earth-centered it was  Geocentric Theory proposed by greeks under the guidance of Ptolemy and Sun-centered was Heliocentric theory proposed by Nicolas Copernicus in 16th century A.D. In short, Your Answers are: 1st blank - Geo-Centric Theory. 2nd blank - Heliocentric Theory.
 2 2593796         13.93430 Copernicus proposed a heliocentric model of the solar system â a model where everything orbited around the Sun. Today, with advancements in science and technology, the geocentric model seems preposterous.he geocentric model, also known as the Ptolemaic system, is a theory that was developed by philosophers in Ancient Greece and was named after the philosopher Claudius Ptolemy who lived circa 90 to 168 A.D. It was developed to explain how the planets, the Sun, and even the stars orbit around the Earth.
 3 6217200         13.93430 The geocentric model, also known as the Ptolemaic system, is a theory that was developed by philosophers in Ancient Greec

In [7]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [8]:
from pygaggle.rerank.transformer import MonoT5

# This step loads the weights of model `castorini/monot5-base-msmarco`, available at HuggingFace's model hub.
reranker = MonoT5() 

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

In [9]:
reranked = reranker.rerank(query, texts)

print('Reranked:')
# Print out reranked results:
for i, text in enumerate(reranked):
    content = json.loads(text.text)['contents']
    print(f'{i+1:2} {text.metadata["docid"]:15} {text.score:.5f} {content}')

Reranked:
 1 7744105         -0.00887 For Earth-centered it was  Geocentric Theory proposed by greeks under the guidance of Ptolemy and Sun-centered was Heliocentric theory proposed by Nicolas Copernicus in 16th century A.D. In short, Your Answers are: 1st blank - Geo-Centric Theory. 2nd blank - Heliocentric Theory.
 2 6217200         -0.01464 The geocentric model, also known as the Ptolemaic system, is a theory that was developed by philosophers in Ancient Greece and was named after the philosopher Claudius Ptolemy who lived circa 90 to 168 A.D. It was developed to explain how the planets, the Sun, and even the stars orbit around the Earth.opernicus proposed a heliocentric model of the solar system â a model where everything orbited around the Sun. Today, with advancements in science and technology, the geocentric model seems preposterous.
 3 264181          -0.02709 Nicolaus Copernicus (b. 1473âd. 1543) was the first modern author to propose a heliocentric theory of the universe.